# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. 

Постройте модель с максимально большим значением *accuracy*. Нужно довести долю правильных ответов по крайней мере до 0.75.

## Откройте и изучите файл

In [4]:
import pandas as pd

df = pd.read_csv("./users_behavior.csv")

print(df.shape)
df.head()

(3214, 5)


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


## Разбейте данные на выборки

In [9]:
from sklearn.model_selection import train_test_split

features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

f_left, f_test, t_left, t_test = train_test_split(features, target, test_size=0.2)
f_train, f_valid, t_train, t_valid = train_test_split(f_left, t_left, test_size=0.25)

## Исследуйте модели

### Модель дерева решений

In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

best_result = 0
best_depth = 0
best_tree_model = None

for depth in range(1, 10):
    model = DecisionTreeClassifier(random_state=12, max_depth=depth)
    model.fit(f_train, t_train)
    predicted = model.predict(f_valid)
    result = accuracy_score(predicted, t_valid)
    if result > best_result:
        best_depth = depth
        best_result = result
        best_tree_model = model

print('Лучший результат: {} при глубине {}'.format(best_result, best_depth))

Лучший результат: 0.7962674961119751 при глубине 7


### Модель случайного леса

In [15]:
from sklearn.ensemble import RandomForestClassifier

best_result = 0
best_est = 0
best_depth = 0
best_rand_tree_model = None

for est in range(10, 51, 5):
    for depth in range(1, 10):
        model = RandomForestClassifier(random_state=12, 
                                       n_estimators=est,
                                       max_depth=depth)
        model.fit(f_train, t_train)
        predicted = model.predict(f_valid)
        result = accuracy_score(predicted, t_valid)
        if result > best_result:
            best_depth = depth
            best_est = est
            best_result = result
            best_rand_tree_model = model

print('Лучший результат:', best_result, 'при количестве деревьев',
      best_est, 'и глубине', best_depth)

Лучший результат: 0.8211508553654744 при количестве деревьев 20 и глубине 8


### Логистическая модель

In [16]:
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression(random_state=12, solver='lbfgs', max_iter=1000)
logistic_model.fit(f_train, t_train)
prediction = logistic_model.predict(f_valid)
result = accuracy_score(prediction, t_valid)
print('Результат:', result)

Результат: 0.7247278382581649


## Проверьте модель на тестовой выборке

In [25]:
# лучшая модель - модель случайного леса
prediction = best_rand_tree_model.predict(f_test)
result = accuracy_score(prediction, t_test)
print('Результат лучшей модели:', result)

Результат лучшей модели: 0.7962674961119751


In [26]:
# проверим на логистической
prediction = logistic_model.predict(f_test)
result = accuracy_score(prediction, t_test)
print('Результат логистической модели:', result)

Результат логистической модели: 0.6951788491446346


In [27]:
# проверим на модели дерева решений
prediction = best_tree_model.predict(f_test)
result = accuracy_score(prediction, t_test)
print('Результат дерева решений:', result)

Результат дерева решений: 0.7947122861586314


## Проверьте модели на адекватность

In [34]:
from sklearn.dummy import DummyClassifier

dummy = DummyClassifier(strategy='most_frequent')
dummy.fit(f_train, t_train)
prediction = dummy.predict(f_valid)
result = accuracy_score(prediction, t_valid)
print(result)

0.7045101088646968


## Вывод

Результат предсказаний логистичесской модели не лучше, чем результат наивного классификатора

Результаты на тестовой выборке у модели случайного леса и модели решающего дерева практически одинаковы, хотя на валидационной выборке точность модели случайного леса была выше, что свидетельствует о ее переобучении

С помощью моделей случайного леса и решающего дерева удалось достичь поставленной точности в 0.75